In [1]:
import DefModules as DM
from datetime import datetime, timedelta
FullOpt =  True
from pgmpy.inference import VariableElimination
import jupyter_contrib_nbextensions
import random
import warnings
import sys 
import logging
import pandas as pd
import time
import numpy as np
import pickle
from tqdm import tqdm
import psycopg2 as pg
import sqlalchemy as sq
import networkx as nx
logging.disable()
if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [2]:
def open_connection():
    '''
    FUNCTION TO CONNECT TO THE POSTGRESQL DATABASE
    '''
    conn = pg.connect(dbname='postgres', user = 'postgres', password = 123, host = 'localhost')
    return conn
def get_connection():
    '''
    FUNCTION TO CONNECT TO THE POSTGRESQL DATABASE AND RETURN THE SQLACHEMY ENGINE OBJECT
    -----------
    output: object
        SQLACHEMY ENGINE OBJECT - POSTGRESQL DATABASE CONNECTION
    '''
    user = 'postgres'
    password = 123
    host = 'localhost'
    port = 5432
    database = 'postgres'
    return sq.create_engine(url="postgresql://{0}:{1}@{2}:{3}/{4}".format(user, password, host, port, database))

In [3]:
def blanket(model,variable):
    '''
    Function to extract the Markov Blanket of the target variable (reduces the structure)
    -----------
    input:
    model: list
        list of edges
        
    variable: str
        name of the target variable
        
    output:
    blanket: list
        list of edges 
    '''
    blanket=[]
    sons=[]
    for i in model:
        if i[0]==variable or i[1]==variable:
            blanket.append(i)
        if i[0]==variable:
            sons.append(i[1])
    for i in model:
        if i[1] in sons and i[0]!=variable:
            blanket.append(i)
    return blanket

In [4]:
def verifica_remove_ciclos(edges):
    '''
    Function to verify if the edges is a DAG and to try remove cycles
    -----------
    input:
    edges: list
        list of edges
                
    output:
    blanket: list
        list of edges 
    '''
    edgesdag = edges #recebe o próprio modelo
    #Verifica se tem ciclos e tenta remover invertendo uma aresta
    if ~nx.is_directed_acyclic_graph(nx.DiGraph(edgesdag[:])):
        for i in edgesdag:  # (3) flip single edge
            edges2 = edgesdag.copy()
            edges2.extend([i[::-1]])
            new_edges = edges2.copy()
            new_edges.remove(i)
            if nx.is_directed_acyclic_graph(nx.DiGraph(new_edges[:])):
                edgesdag = new_edges.copy()
                break
    #Verifica se tem ciclos e tenta remover invertendo duas arestas
    if ~nx.is_directed_acyclic_graph(nx.DiGraph(edgesdag[:])):
        for i in edgesdag:
            for j in edgesdag:# (3) flip two edges
                if i != j:
                    edges2 = edgesdag.copy()
                    edges2.extend([i[::-1]])
                    edges2.extend([j[::-1]])
                    new_edges = edges2.copy()
                    new_edges.remove(i)
                    new_edges.remove(j)
                    if nx.is_directed_acyclic_graph(nx.DiGraph(new_edges[:])):
                        edgesdag = new_edges.copy()
                        breaker = True
                        break
            if breaker:
                break
    #Verifica se tem ciclos e tenta remover invertendo uma aresta e excluindo uma aresta
    if ~nx.is_directed_acyclic_graph(nx.DiGraph(edgesdag[:])):
        for i in edgesdag:
            for j in edgesdag:# (3) flip two edges
                if i != j:
                    edges2 = edgesdag.copy()
                    edges2.extend([i[::-1]])
                    new_edges = edges2.copy()
                    new_edges.remove(i)
                    new_edges.remove(j)
                    if nx.is_directed_acyclic_graph(nx.DiGraph(new_edges[:])):
                        edgesdag = new_edges.copy()
                        breaker = True
                        break
            if breaker:
                break
    return edgesdag

In [5]:
def get_all_dates(pais):
    q = '''select distinct cast("Date" as DATE) as datas from pre_processed_data.dbn_features_selected_{pais} order by datas'''.format(pais=pais)
    conn = open_connection()
    date = pd.read_sql(q,conn)
    conn.close()
    datas = date['datas'].tolist()
    return datas

In [6]:
def get_dataset(pais,date_ini, date_fin):
    q = '''select * 
    from pre_processed_data.dbn_features_selected_{pais} where "Date" between '{date_ini}' and '{date_fin}' '''.format(pais=pais,date_ini=date_ini,date_fin=date_fin)
    conn = open_connection()
    dataset = pd.read_sql(q,conn)
    conn.close()
    return dataset

In [7]:
def update_edges_frequencies(best_model, edges_possibilities, edges_frequency):
    if not edges_possibilities:
        edges_possibilities = best_model
        for p in range(len(edges_possibilities)):
            edges_frequency.append(1)
    else:
        for v in range(len(best_model)):
            if best_model[v] not in edges_possibilities:
                edges_possibilities.append(best_model[v])
                edges_frequency.append(1)
            else:
                for f in range(len(edges_possibilities)):
                    if best_model[v] == edges_possibilities[f]:
                        edges_frequency[f]=edges_frequency[f]+1
    return edges_possibilities, edges_frequency

In [44]:
def update_threshold_select_edges(k, edges_possibilities, edges_frequency):
    fth = 1/3+np.sqrt(2/k)
    if fth>0.4:
        fth=0.4
    edges_frequency_v=[edges_frequency[i]/k for i in range(len(edges_frequency))]
    edges=[]
    for i in range(len(edges_possibilities)):
        if edges_frequency_v[i]>=fth and edges_possibilities[i] not in edges:
            if edges_possibilities[i][::-1] not in edges_possibilities:
                edges.append(edges_possibilities[i])
            else: 
                if edges_frequency_v[i] > edges_frequency_v[edges_possibilities.index(edges_possibilities[i][::-1])]:
                    edges.append(edges_possibilities[i])
                else: 
                    edges.append(edges_possibilities[i][::-1])
        elif edges_frequency_v[i]<fth:
            for j in range(len(edges_possibilities)):
                if edges_possibilities[i]==edges_possibilities[j][::-1]:
                    if edges_frequency_v[i]+edges_frequency_v[j]>=fth:
                        if edges_frequency_v[i]>edges_frequency_v[j]:
                            edges.append(edges_possibilities[i])
                        if edges_frequency_v[i]<edges_frequency_v[j]:
                            edges.append(edges_possibilities[j])
                        if edges_frequency_v[i]==edges_frequency_v[j]:
                            auxci=0
                            auxcj=0
                            for s in range(len(edges)):
                                if edges[s]==edges_possibilities[i]:
                                    auxci=auxci+1
                                if edges[s]==edges_possibilities[j]:
                                    auxcj=auxcj+1
                            if auxci>0:
                                edges.append(edges_possibilities[i])
                            elif auxcj>0:
                                edges.append(edges_possibilities[j])
                            else: 
                                import random
                                edges.append(random.choice([edges_possibilities[i],edges_possibilities[j]]))
    edges = list(set(edges))
    return edges

In [9]:
def main(pais):
    #initialize auxiliary variables
    k=1 #total days used
    target_variable = 'Emission'
    edges_possibilities = []
    edges_frequency = []
    timemodel = []
    timeinference = []
    
    #read all available dates
    dates = get_all_dates(pais)
    
    #begin the forecast experiment
    for i in tqdm(dates[0:3]):
        #dataset to learn the model
        data_learn = get_dataset(pais,i, i+timedelta(days = 0))
        #structural learning with the dataset of day i
        data_learn.drop(['Date','Hour'], axis = 1, inplace = True)
        ti = time.time()
        best_model = DM.EdgesModel(data_learn, FullOpt)[0]
        
        #get the markov blanket
        best_model = blanket(best_model, target_variable)
        
        #update the edges frequencies
        edges_possibilities, edges_frequency = update_edges_frequencies(best_model, edges_possibilities, edges_frequency)
        
        #update threshold and select the edges
        edges = update_threshold_select_edges(k, edges_possibilities, edges_frequency)
        
        tf = time.time()
        timemodel.append(tf-ti)
        #forecast initial in day 8 (fit from 01 until 07)
        if i >= dates[6]:
            #fit dataset (last 7 days)
            data_fit = get_dataset(pais,i-timedelta(days = 6), i)
            
            #predict data of the entire day
            predict_data_day = get_dataset(pais,i+timedelta(days = 1), i+timedelta(days = 1))
            
        k = k+1
            
    return edges_possibilities, edges_frequency, k, timemodel    

In [49]:
dates = dates = get_all_dates('alemanha')
i = dates[6]
predict_data_day = get_dataset('alemanha',i+timedelta(days = 1), i+timedelta(days = 1))

In [51]:
i

datetime.date(2019, 1, 7)

In [50]:
predict_data_day

,Date,Hour,Emission,Emission-1,Lignite,Lignite-1,Hard coal,Hard coal-1,W. Onshore,Fossil Gas,W. Onshore-1,Fossil Gas-1,Nuclear,Nuclear-1,Solar,W. Offshore,W. Offshore-1
0,2019-01-08,0,6,9,9,15,5,12,30,6,29,11,31,34,0,14,16
1,2019-01-08,1,5,8,9,11,5,9,30,6,29,10,33,34,0,14,15
2,2019-01-08,2,5,7,9,10,5,6,30,6,30,9,32,34,0,14,15
3,2019-01-08,3,5,6,9,9,5,5,30,6,30,6,33,31,0,14,14
4,2019-01-08,4,6,5,9,9,5,5,30,7,30,6,33,33,0,13,14
5,2019-01-08,5,6,5,10,9,6,5,30,8,30,6,34,32,0,12,14
6,2019-01-08,6,7,5,11,9,9,5,32,10,30,6,34,33,0,12,14
7,2019-01-08,7,8,6,14,9,10,5,33,11,30,7,34,33,0,11,13
8,2019-01-08,8,9,6,16,10,11,6,33,12,30,8,34,34,0,10,12
9,2019-01-08,9,8,7,16,11,10,9,34,12,32,10,34,34,0,12,12


In [10]:
edges_possibilities, edges_frequency, k, timemodel = main('alemanha')

100%|████████████████████████████████████████████████████████████████████████████████| 3/3 [1:39:50<00:00, 1996.81s/it]


In [21]:
edges_possibilities[0][::-1]

('W. Offshore', 'Emission')

In [22]:
edges_possibilities[0]

('Emission', 'W. Offshore')

In [18]:
edges_frequency_v=[edges_frequency[i]/k for i in range(len(edges_frequency))]

In [17]:
edges_frequency

[2, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1]

In [19]:
edges_frequency_v

[0.5,
 0.5,
 0.5,
 0.25,
 0.25,
 0.5,
 0.25,
 0.5,
 0.5,
 0.25,
 0.25,
 0.25,
 0.25,
 0.25,
 0.25]

In [68]:
#
data_learn.head()

,Emission,Emission-1,Lignite,Lignite-1,Hard coal,Hard coal-1,W. Onshore,Fossil Gas,W. Onshore-1,Fossil Gas-1,Nuclear,Nuclear-1,Solar,W. Offshore,W. Offshore-1
0,7,16,8,22,4,12,19,4,15,6,22,23,0,23,23
1,6,14,5,20,4,10,19,5,16,5,21,23,0,25,23
2,5,11,4,16,4,6,19,4,18,5,21,22,0,25,22
3,6,7,4,8,4,4,18,4,19,4,19,22,0,23,23
4,6,6,4,5,4,4,18,5,19,5,19,21,0,23,25


In [69]:
data_learn.drop(['Date','Hour'], axis = 1, inplace = True)
best_model = DM.EdgesModel(data_learn, FullOpt)[0]

In [70]:
best_model

[('Emission', 'Lignite'),
 ('Emission', 'W. Offshore-1'),
 ('Emission-1', 'Nuclear-1'),
 ('Hard coal', 'W. Offshore'),
 ('Hard coal', 'Fossil Gas'),
 ('Hard coal-1', 'Nuclear'),
 ('Hard coal-1', 'Emission'),
 ('Hard coal-1', 'W. Onshore-1'),
 ('Hard coal-1', 'Lignite-1'),
 ('Hard coal-1', 'W. Offshore'),
 ('Hard coal-1', 'Lignite'),
 ('Fossil Gas', 'Emission'),
 ('Fossil Gas', 'Lignite-1'),
 ('W. Onshore-1', 'Lignite'),
 ('Fossil Gas-1', 'Nuclear-1'),
 ('Nuclear', 'Hard coal'),
 ('Nuclear', 'W. Onshore'),
 ('Nuclear-1', 'Hard coal-1'),
 ('Nuclear-1', 'Fossil Gas'),
 ('Solar', 'Hard coal-1'),
 ('Solar', 'W. Offshore-1'),
 ('Solar', 'Emission'),
 ('Solar', 'Nuclear'),
 ('Solar', 'Lignite')]

In [74]:
edges_possibilities = []
edges_frequency = []
best_model = blanket(best_model, 'Emission')

In [75]:
edges_possibilities, edges_frequency = update_edges_frequencies(best_model, edges_possibilities, edges_frequency)

In [76]:
edges_possibilities

[('Emission', 'Lignite'),
 ('Emission', 'W. Offshore-1'),
 ('Hard coal-1', 'Emission'),
 ('Fossil Gas', 'Emission'),
 ('Solar', 'Emission'),
 ('Hard coal-1', 'Lignite'),
 ('W. Onshore-1', 'Lignite'),
 ('Solar', 'W. Offshore-1'),
 ('Solar', 'Lignite')]

In [77]:
edges_frequency

[1, 1, 1, 1, 1, 1, 1, 1, 1]

In [86]:
best_model = [('Emission', 'Lignite'),
 ('Emission', 'W. Offshore-1'),
 ('Hard coal-1', 'Emission'),
 ('Fossil Gas', 'Emission'),
 ('Solar', 'Emission'),
 ('Hard coal-1', 'Lignite'),
 ('W. Onshore-1', 'Lignite'),
 ('Solar', 'W. Offshore-1'),
 ('Solar', 'Lignite'), ('k','k2'), ('t1','t3')]

In [88]:
edges_possibilities

[('Emission', 'Lignite'),
 ('Emission', 'W. Offshore-1'),
 ('Hard coal-1', 'Emission'),
 ('Fossil Gas', 'Emission'),
 ('Solar', 'Emission'),
 ('Hard coal-1', 'Lignite'),
 ('W. Onshore-1', 'Lignite'),
 ('Solar', 'W. Offshore-1'),
 ('Solar', 'Lignite'),
 ('k', 'k2'),
 ('t1', 't3')]

In [89]:
edges_frequency

[4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 1]

In [87]:
edges_possibilities, edges_frequency = update_edges_frequencies(best_model, edges_possibilities, edges_frequency)

In [14]:
q = '''select distinct "Date" from pre_processed_data.dbn_features_selected_alemanha order by "Date"'''
conn = open_connection()
date = pd.read_sql(q,conn)
conn.close()
date['Date'] = date['Date']+timedelta(days = 0)
date.head()

,Date
0,2019-01-01
1,2019-01-02
2,2019-01-03
3,2019-01-04
4,2019-01-05


In [15]:
date['Date'][7]

Timestamp('2019-01-08 00:00:00')

In [12]:
date['Date']+timedelta(days = 0)

0      2019-01-01
1      2019-01-02
2      2019-01-03
3      2019-01-04
4      2019-01-05
          ...    
1086   2021-12-26
1087   2021-12-27
1088   2021-12-28
1089   2021-12-29
1090   2021-12-30
Name: Date, Length: 1091, dtype: datetime64[ns]

In [41]:
edges_possibilities = [('x','y'),('y','x')]
edges_frequency_v = [0.9,0.7]
fth = 0.6

In [42]:
edges=[]
for i in range(len(edges_possibilities)):
    if edges_frequency_v[i]>=fth and edges_possibilities[i] not in edges:
        if edges_possibilities[i][::-1] not in edges_possibilities:
            edges.append(edges_possibilities[i])
        else: 
            if edges_frequency_v[i] > edges_frequency_v[edges_possibilities.index(edges_possibilities[i][::-1])]:
                edges.append(edges_possibilities[i])
            else: 
                edges.append(edges_possibilities[i][::-1])
    elif edges_frequency_v[i]<fth:
        for j in range(len(edges_possibilities)):
            if edges_possibilities[i]==edges_possibilities[j][::-1]:
                if edges_frequency_v[i]+edges_frequency_v[j]>=fth:
                    if edges_frequency_v[i]>edges_frequency_v[j]:
                        edges.append(edges_possibilities[i])
                    if edges_frequency_v[i]<edges_frequency_v[j]:
                        edges.append(edges_possibilities[j])
                    if edges_frequency_v[i]==edges_frequency_v[j]:
                        auxci=0
                        auxcj=0
                        for s in range(len(edges)):
                            if edges[s]==edges_possibilities[i]:
                                auxci=auxci+1
                            if edges[s]==edges_possibilities[j]:
                                auxcj=auxcj+1
                        if auxci>0:
                            edges.append(edges_possibilities[i])
                        elif auxcj>0:
                            edges.append(edges_possibilities[j])
                        else: 
                            import random
                            edges.append(random.choice([edges_possibilities[i],edges_possibilities[j]]))
edges = list(set(edges))

In [43]:
edges

[('x', 'y')]